
# PyTorch Tabular Binary Classifier (MLP) — Loan Default (`bad_flag`)

This notebook trains a **Multi-Layer Perceptron (MLP)** on your tabular dataset to predict `bad_flag` (binary target).

> **Data assumption:** Rows with `bad_flag` = NaN are treated as **test** rows for inference.



## 0) Requirements
Run this once to install dependencies (if needed):


In [1]:

# If running locally and you need packages, uncomment:
#!pip install torch scikit-learn pandas joblib openpyxl


## 1) Imports & Reproducibility

In [2]:

import os, json, random, math
from typing import List, Tuple

import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_fscore_support

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED = 42
set_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

## 2) Load Dataset

In [3]:

# Update the path if needed
DATA_PATH = r"C:\Users\luwil\Documents\misc_data\modeldata.xlsx"

df = pd.read_excel(DATA_PATH)
print(df.shape)
df.head()


(291962, 27)


,Unnamed: 0,id,member_id,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,desc,...,total_bc_limit,mths_since_last_major_derog,tot_hi_cred_lim,tot_cur_bal,application_approved_flag,internal_score,bad_flag,emp_length_num,desc_length,has_desc
0,0,10000001,11983056,7550,36,16.24,3 years,RENT,28000.0,NaN,...,4000.0,NaN,3828.953801,5759.0,1,99,0.0,3.0,0,0
1,1,10000002,12002921,27050,36,10.99,10+ years,OWN,55000.0,Borrower added on 12/31/13 > Combining high ...,...,35700.0,NaN,34359.940730,114834.0,1,353,0.0,10.0,95,1
2,2,10000003,11983096,12000,36,10.99,4 years,RENT,60000.0,Borrower added on 12/31/13 > I would like to...,...,18100.0,NaN,16416.617760,7137.0,1,157,0.0,4.0,176,1
3,3,10000004,12003142,28000,36,7.62,5 years,MORTGAGE,325000.0,NaN,...,42200.0,NaN,38014.149760,799592.0,1,365,0.0,5.0,0,0
4,4,10000005,11993233,12000,36,13.53,10+ years,RENT,40000.0,NaN,...,7000.0,53.0,6471.462236,13605.0,1,157,0.0,10.0,0,0


## 3) Preprocessing Utilities - Data cleaning to drop unneeded columns, impute missing values, categorical one hot encoding, scaling numeric features

In [4]:

DROP_COLS_DEFAULT = ["Unnamed: 0", "member_id", "desc"]   # obvious non-features
POSSIBLE_DROP_IF_PRESENT = ["emp_length"]                 # keep emp_length_num instead if present

def preprocess(df: pd.DataFrame, target_col: str = "bad_flag"):
    df = df.copy()

    # Drop obvious columns if present
    for c in DROP_COLS_DEFAULT + POSSIBLE_DROP_IF_PRESENT:
        if c in df.columns:
            df.drop(columns=[c], inplace=True)

    # Identify test rows (bad_flag missing)
    is_test = df[target_col].isna() if target_col in df.columns else pd.Series([False] * len(df))
    train_df = df.loc[~is_test].copy()
    test_df  = df.loc[is_test].copy()

    # Target
    y = None
    if target_col in train_df.columns:
        y = train_df[target_col].astype(int)
        train_df.drop(columns=[target_col], inplace=True)
    if target_col in test_df.columns:
        test_df.drop(columns=[target_col], inplace=True)

    # Separate types
    cat_cols = train_df.select_dtypes(include=["object", "category"]).columns.tolist()
    num_cols = train_df.select_dtypes(include=["number"]).columns.tolist()

    # Keep ID for output but not as feature
    id_col = None
    for cand in ["id", "ID", "Id"]:
        if cand in train_df.columns:
            id_col = cand
            if cand in num_cols: 
                num_cols.remove(cand)
            break

    # Impute missing
    for c in cat_cols:
        train_df[c] = train_df[c].fillna("Unknown")
    for c in num_cols:
        med = train_df[c].median() # we use median here as it is robust to outliers.
        train_df[c] = train_df[c].fillna(med)
    # impute missing using the training portion of the dataset to prevent data leakage
    if len(test_df) > 0:
        for c in cat_cols:
            if c in test_df.columns:
                test_df[c] = test_df[c].fillna("Unknown")
        for c in num_cols:
            if c in test_df.columns:
                med = train_df[c].median()
                test_df[c] = test_df[c].fillna(med)

    # One-hot encode categoricals across combined frame to align columns
    combined = pd.concat([train_df, test_df], axis=0, sort=False)
    combined = pd.get_dummies(combined, columns=cat_cols, drop_first=True)

    # Split back
    X_all   = combined
    X_train = X_all.iloc[: len(train_df)].copy()
    X_test  = X_all.iloc[len(train_df):].copy()

    # Scale numeric columns only
    scaler = StandardScaler()
    if num_cols:
        scaler.fit(X_train[num_cols].values)
        X_train.loc[:, num_cols] = scaler.transform(X_train[num_cols].values)
        if len(X_test) > 0:
            X_test.loc[:, num_cols]  = scaler.transform(X_test[num_cols].values)

    feature_cols = X_train.columns.tolist()
    return X_train, y, X_test, feature_cols, scaler, id_col
 

## 4) Run Preprocessing

In [5]:

X_train_df, y, X_test_df, feature_cols, scaler, id_col = preprocess(df, target_col="bad_flag")
print("Train shape:", X_train_df.shape, "Test shape:", X_test_df.shape, "Features:", len(feature_cols))
y.value_counts(normalize=True).rename("class_ratio")


Train shape: (189457, 36) Test shape: (102505, 36) Features: 36


C:\Users\luwil\AppData\Local\Temp\ipykernel_19588\717782101.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.83870292  1.56511359 -0.29013967 ...  0.2029509  -1.02977552
  1.6822226 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train.loc[:, num_cols] = scaler.transform(X_train[num_cols].values)
C:\Users\luwil\AppData\Local\Temp\ipykernel_19588\717782101.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.55534196 -0.55534196 -0.55534196 ...  1.80069231 -0.55534196
  1.80069231]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train.loc[:, num_cols] = scaler.transform(X_train[num_cols].values)
C:\Users\luwil\AppData\Local\Temp\ipykernel_19588\717782101.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in 

bad_flag
0    0.930707
1    0.069293
Name: class_ratio, dtype: float64

### define MLP class and dataset type class

In [10]:
#Tabular dataset is a function to make our dataset able to be used with pytorch's dataloader function
class TabularDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray | None = None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        return self.X[idx], self.y[idx]

class MLP(nn.Module):
    def __init__(self, in_dim: int, hidden: List[int], dropout: float = 0.2, use_bn: bool = True):
        super().__init__()
        layers: List[nn.Module] = []
        prev = in_dim
        for h in hidden:
            layers.append(nn.Linear(prev, h))
            if use_bn:
                layers.append(nn.BatchNorm1d(h))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(p=dropout))
            prev = h
        layers.append(nn.Linear(prev, 1))  # binary logit
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x).squeeze(-1)  # logits

#%% md
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for Xb, yb in loader:
        Xb = Xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * Xb.size(0)
    return running_loss / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    all_logits = []
    all_targets = []
    running_loss = 0.0
    for Xb, yb in loader:
        Xb = Xb.to(device)
        yb = yb.to(device)
        logits = model(Xb)
        loss = criterion(logits, yb)
        running_loss += loss.item() * Xb.size(0)
        all_logits.append(logits.detach().cpu().numpy())
        all_targets.append(yb.detach().cpu().numpy())
    avg_loss = running_loss / len(loader.dataset)

    logits = np.concatenate(all_logits)
    targets = np.concatenate(all_targets).astype(int)
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= 0.5).astype(int)

    try:
        auc = roc_auc_score(targets, probs)
    except ValueError:
        auc = float('nan')

    acc = accuracy_score(targets, preds)
    pr, rc, f1, _ = precision_recall_fscore_support(targets, preds, average="binary", zero_division=0)
    return {"loss": avg_loss, "auc": auc, "acc": acc, "precision": pr, "recall": rc, "f1": f1}

class EarlyStopper:
    def __init__(self, patience: int = 5, mode: str = "max"):
        self.patience = patience
        self.mode = mode
        self.best = -float("inf") if mode == "max" else float("inf")
        self.count = 0
        self.best_state = None

    def step(self, metric_value, model):
        improved = (metric_value > self.best) if self.mode == "max" else (metric_value < self.best)
        if improved:
            self.best = metric_value
            self.count = 0
            self.best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            return True
        else:
            self.count += 1
            return False

    def should_stop(self):
        return self.count >= self.patience


### create train/validation split data

In [13]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_df.astype(np.float32).values, 
    y.astype(np.float32).values,
    test_size=0.2,
    random_state=SEED,
    stratify=y.values
)

train_ds = TabularDataset(X_train, y_train)
val_ds   = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=512, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=512, shuffle=False, num_workers=0)